## Code 1

In [ ]:
import pandas as pd
import torch
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def predict_extremophiles(sentences):
    # Tokenize and encode the sentences
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted probabilities for each class
    predicted_probabilities = torch.softmax(outputs.logits, dim=1)

    # Threshold probability for classification
    threshold = 0.5

    predictions = []
    # Classify sentences based on the threshold
    for i, sentence in enumerate(sentences):
        extremophile_probability = predicted_probabilities[i][1].item()
        if extremophile_probability > threshold:
            predictions.append((sentence, extremophile_probability, True))
        else:
            predictions.append((sentence, extremophile_probability, False))
    return predictions

# Fix the random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import random
random.seed(random_seed)
os.environ['PYTHONHASHSEED'] = str(random_seed)

# Suppress warning messages
os.environ['PYTHONWARNINGS'] = "ignore"

# Load pre-trained BioBERT model and tokenizer
model_name = "monologg/biobert_v1.1_pubmed"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the CSV file
csv_path = '/content/Task4_Genes_give_sentence.csv'
chunk_size = 100

# Create an empty list to store all predictions
all_predictions = []

# Iterate through the CSV file in chunks of 100 rows
for df_chunk in pd.read_csv(csv_path, chunksize=chunk_size):
    # Extract the column of sentences from the current chunk
    sentences_column = df_chunk['Sentences']
    # Convert the column to a list of sentences
    sentences_list = sentences_column.tolist()
    # Predict extremophiles for the sentences
    predictions = predict_extremophiles(sentences_list)
    # Store predictions in the list
    all_predictions.extend(predictions)

# Convert predictions to DataFrame
result_df = pd.DataFrame(all_predictions, columns=['Sentences', 'Probability', 'Verify'])

# Save the results to a new CSV file
result_csv_path = '/content/Genes_Extremophiles_results.csv'
result_df.to_csv(result_csv_path, index=False)

print("Results saved to:", result_csv_path)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/biobert_v1.1_pubmed and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Results saved to: /content/Genes_Extremophiles_results.csv


## Code 2

In [ ]:
import pandas as pd
import torch
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

# Load pre-trained BioBERT model and tokenizer
model_name = "monologg/biobert_v1.1_pubmed"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the CSV file
csv_path = '/content/drive/MyDrive/Task4_Genes_give_sentence.csv'
extremophiles_df = pd.read_csv(csv_path)

# Prepare your labeled dataset for fine-tuning
# Assuming 'Sentences' column contains sentences related to extremophiles

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(extremophiles_df, test_size=0.1, random_state=42)

# Tokenize and encode the training and validation sets
train_encodings = tokenizer(train_df['Sentences'].tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(val_df['Sentences'].tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')

# Define labels (assuming all sentences in task4 are related to extremophiles)
train_labels = torch.ones(len(train_df)).long()  # All sentences are extremophiles
val_labels = torch.ones(len(val_df)).long()

# Define datasets
class ExtremophilesDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ExtremophilesDataset(train_encodings, train_labels)
val_dataset = ExtremophilesDataset(val_encodings, val_labels)

# Define output directory
output_dir = "/content/drive/MyDrive/fine_tuned_model"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs',
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()

# Save the model
model.save_pretrained(output_dir)

print("Model trained and saved at:", output_dir)

# Evaluate the model
eval_results = trainer.evaluate()

print("Evaluation Results:", eval_results)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/biobert_v1.1_pubmed and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-1-ff4ee27879de>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires

Model trained and saved at: /content/drive/MyDrive/fine_tuned_model


Evaluation Results: {'eval_loss': 3.2188676414079964e-05, 'eval_runtime': 334.4791, 'eval_samples_per_second': 0.664, 'eval_steps_per_second': 0.084, 'epoch': 1.0}


In [ ]:
import pandas as pd
import torch
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

# Load pre-trained BioBERT model and tokenizer
model_name = "monologg/biobert_v1.1_pubmed"
tokenizer = AutoTokenizer.from_pretrained(model_name)
output_dir = "/content/drive/MyDrive/fine_tuned_model"
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/fine_tuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_model/vocab.txt',
 '/content/drive/MyDrive/fine_tuned_model/added_tokens.json',
 '/content/drive/MyDrive/fine_tuned_model/tokenizer.json')

In [ ]:
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the saved model and tokenizer
model_path = '/content/drive/MyDrive/fine_tuned_model'  # Path to your saved model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Function to perform predictions on new data
def predict_extremophiles(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_probabilities = torch.softmax(outputs.logits, dim=1)
    threshold = 0.5
    predictions = [(sentence, prob.item(), prob.item() > threshold) for sentence, prob in zip(sentences, predicted_probabilities[:, 1])]
    return predictions

# Load the CSV file for predictions (if needed)
csv_path = '/content/drive/MyDrive/Task4_Genes_give_sentence.csv'
chunk_size = 100
all_predictions = []

# Iterate through the CSV file in chunks
for df_chunk in pd.read_csv(csv_path, chunksize=chunk_size):
    sentences_list = df_chunk['Sentences'].tolist()
    predictions = predict_extremophiles(sentences_list)
    all_predictions.extend(predictions)

# Convert predictions to DataFrame
result_df = pd.DataFrame(all_predictions, columns=['Sentences', 'Probability', 'Verify'])

# Save the results to a new CSV file
result_csv_path = '/content/drive/MyDrive/Genes_Extremophiles_results.csv'
result_df.to_csv(result_csv_path, index=False)

print("Results saved to:", result_csv_path)


Results saved to: /content/drive/MyDrive/Genes_Extremophiles_results.csv
